In [34]:
!pip install -qU langchain langchain-anthropic bs4 anthropic

In [1]:
import getpass
anthropic_api_key = getpass.getpass("Anthropic API key: ")


In [12]:
model ="claude-3-5-sonnet-latest"
wait_time_seconds = 60

In [13]:
import requests
import os

from bs4 import BeautifulSoup

# Text of the U.S. Constitution - about 66k tokens!
url = "https://www.govinfo.gov/content/pkg/CDOC-110hdoc50/html/CDOC-110hdoc50.htm" 
txt_file_name = "constitution.txt"

if os.path.isfile(txt_file_name):
    with open(txt_file_name, "r", encoding="utf-8") as file:
        us_constitution_text = file.read()
else:
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    us_constitution_text = soup.get_text(separator="\n", strip=True)
        
    # Save the text content to a .txt file
    output_file = txt_file_name
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(text_content)

In [3]:
test_questions = [
    "What six goals are outlined in the Preamble of the U.S. Constitution?",
    "How does Article I define the composition and powers of the House and Senate?",
    "What process does Article V establish for proposing and ratifying constitutional amendments?",
    "What powers are explicitly granted to Congress under Article I, Section 8?",
    "How does the Constitution address the resolution of disputes between states in Article III?"
]

system_prompt = f"""Answer the question based on the context. 

Context: {us_constitution_text}
"""

In [14]:
import anthropic
import time

client = anthropic.Anthropic(api_key=anthropic_api_key)
for i,question in enumerate(test_questions):
  start_time = time.time()
  response = client.messages.create(
      model=model,
      max_tokens=1024,
      system=[
        {
          "type": "text",
          "text": system_prompt,
        }
      ],
      messages=[{"role": "user", "content": f"Question to analyze: {question}"}],
  )
  print(response.content)
  print(f"Cache usage: {response.usage}")
  print(f"Time: {time.time()-start_time}s")

  if i<(len(test_questions)-1):
    time.sleep(wait_time_seconds) 

[TextBlock(text='According to the Preamble of the U.S. Constitution, the six goals are:\n\n1. "Form a more perfect Union"\n2. "Establish Justice"\n3. "Insure domestic Tranquility"\n4. "Provide for the common defence"\n5. "Promote the general Welfare"\n6. "Secure the Blessings of Liberty to ourselves and our Posterity"\n\nThese goals are laid out in the opening paragraph of the Constitution, which reads:\n\n"We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."\n\nThese six goals represent the fundamental purposes and aspirations that the Constitution was designed to achieve for the United States.', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70549, output_t

In [15]:
for i,question in enumerate(test_questions):
  start_time = time.time()
  response = client.messages.create(
      model=model,
      max_tokens=1024,
      system=[
        {
          "type": "text",
          "text": system_prompt,
          "cache_control": {"type": "ephemeral"}
        }
      ],
      messages=[{"role": "user", "content": f"Question to analyze: {question}"}],
  )
  print(response.content)
  print(f"Cache usage: {response.usage}")
  print(f"Time: {time.time()-start_time}s")

  if i<(len(test_questions)-1):
    time.sleep(wait_time_seconds) 

[TextBlock(text='According to the Preamble of the U.S. Constitution, the six goals outlined are:\n\n1. To form a more perfect Union\n2. To establish Justice\n3. To insure domestic Tranquility\n4. To provide for the common defence\n5. To promote the general Welfare\n6. To secure the Blessings of Liberty to ourselves and our Posterity\n\nThese goals are stated in the opening lines of the Constitution:\n\n"We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."\n\nThese six goals represent the fundamental purposes for which the Constitution was created and serve as guiding principles for the American system of government.', type='text')]
Cache usage: Usage(cache_creation_input_tokens=70520, cache_read_input_tokens=0, inp

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

# Set up a ChatAnthropic LLM without prompt caching, to baseline.
llm = ChatAnthropic(
    model="claude-3-5-haiku-latest",  
    anthropic_api_key=anthropic_api_key  
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "Question: {question}"),
    ]
)

chain = prompt | llm

In [7]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

In [ ]:
import time

for question in test_questions:
    start_time = time.time()
    print(chain.invoke({
            "question": question,
            })   
    )

    print(f"Time: {time.time()-start_time}\n")
    time.sleep(wait_time_seconds) # Tier 2 account, so sleep for a bit before calling again


In [10]:
import time

start_time = time.time()
print(chain.invoke({
    "question": test_questions[0],
    })   
)

print(f"Time: {time.time()-start_time}\n")

content='Based on the Preamble text in the document, the six goals outlined are:\n\n1. To form a more perfect Union\n2. To establish Justice\n3. To insure domestic Tranquility\n4. To provide for the common defence\n5. To promote the general Welfare\n6. To secure the Blessings of Liberty to ourselves and our Posterity\n\nThe full Preamble reads: "We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."' additional_kwargs={} response_metadata={'id': 'msg_01A4DYzJHYCQEPfAnx8AArYZ', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 70547, 'output_tokens': 165}} id='run-3434c0e5-348c-44fc-a8bf-930f

Haiku results:

[TextBlock(text='Based on the context of the Preamble in the document, the six goals outlined are:\n\n1. To form a more perfect Union\n2. To establish Justice\n3. To insure domestic Tranquility\n4. To provide for the common defense\n5. To promote the general Welfare\n6. To secure the Blessings of Liberty to themselves and their Posterity\n\nThis can be seen in the Preamble text: "We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70549, output_tokens=171)
Time: 12.880332708358765s
[TextBlock(text="Based on Article I of the Constitution, here's how the House of Representatives and Senate are defined:\n\nHouse of Representatives Composition:\n- Members chosen every two years by the people of each state\n- Qualifications:\n  * Must be at least 25 years old\n  * Must have been a U.S. citizen for 7 years\n  * Must be an inhabitant of the state they represent\n- Number of Representatives based on state population\n- Initially set at:\n  * New Hampshire: 3\n  * Massachusetts: 8\n  * Rhode Island: 1\n  * Connecticut: 5\n  * New York: 6\n  * New Jersey: 4\n  * Pennsylvania: 8\n  * Delaware: 1\n  * Maryland: 6\n  * Virginia: 10\n  * North Carolina: 5\n  * South Carolina: 5\n  * Georgia: 3\n\nSenate Composition:\n- Originally, two Senators per state chosen by state legislatures\n- Senators serve six-year terms\n- Senators initially divided into three classes with staggered elections\n- Qualifications:\n  * Must be at least 30 years old\n  * Must have been a U.S. citizen for 9 years\n  * Must be an inhabitant of the state they represent\n\nPowers of the House:\n- Sole power of impeachment\n- Chooses its own Speaker and officers\n\nPowers of the Senate:\n- Sole power to try impeachments\n- Chooses its own officers\n- Confirms Presidential appointments\n- Ratifies treaties", type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70545, output_tokens=353)
Time: 16.046749114990234s
[TextBlock(text='Based on Article V in the Constitution, the process for proposing and ratifying constitutional amendments has two primary methods:\n\nProposal Methods:\n1. Congressional Proposal: When two-thirds of both Houses of Congress deem it necessary, Congress can propose amendments\n\n2. State-Initiated Convention: When legislatures of two-thirds of the states (currently 34 states) apply to Congress, Congress must call a convention for proposing amendments\n\nRatification Methods:\nAmendments can be ratified by either:\n1. Legislatures of three-fourths of the states (currently 38 states), OR\n2. Conventions in three-fourths of the states, as Congress may prescribe\n\nKey Limitations in the Original Text:\n- No amendment prior to 1808 could affect certain provisions related to slave importation\n- No state can be deprived of its equal suffrage in the Senate without its consent\n\nThe text emphasizes flexibility in the amendment process while also setting a high bar (broad consensus) for making changes to the Constitution, requiring supermajority support at both the proposal and ratification stages.\n\nThis process ensures that constitutional amendments represent a broad, nationwide consensus rather than being easily manipulated by a simple majority or narrow regional interests.', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70545, output_tokens=270)
Time: 15.6591317653656s
[TextBlock(text='Based on the context provided in Article I, Section 8 of the Constitution, Congress is explicitly granted the following powers:\n\n1. To lay and collect taxes, duties, imposts, and excises (uniformly throughout the United States)\n2. To borrow money on the credit of the United States\n3. To regulate commerce with foreign nations, among the states, and with Indian tribes\n4. To establish a uniform rule of naturalization and uniform bankruptcy laws\n5. To coin money and regulate its value\n6. To provide for punishing counterfeiting\n7. To establish post offices and post roads\n8. To promote the progress of science and useful arts by securing copyrights and patents\n9. To constitute tribunals inferior to the Supreme Court\n10. To define and punish piracies and felonies committed on the high seas\n11. To declare war, grant letters of marque and reprisal, and make rules about captures\n12. To raise and support armies (with appropriations limited to two years)\n13. To provide and maintain a Navy\n14. To make rules for governing land and naval forces\n15. To call forth the militia to execute laws, suppress insurrections, and repel invasions\n16. To provide for organizing, arming, and disciplining the militia\n17. To exercise exclusive legislation over the seat of government and federal properties\n18. To make all laws necessary and proper for executing the foregoing powers and other powers vested in the government', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70545, output_tokens=319)
Time: 16.61333203315735s
[TextBlock(text='Based on the text of Article III in the Constitution, disputes between states are addressed in Section 2, Clause 1, which states that the judicial power of the United States shall extend to "Controversies between two or more States".\n\nSpecifically, the relevant text reads: "The judicial power shall extend to... Controversies between two or more States". This means that the Supreme Court has original jurisdiction in cases where states are in direct conflict with each other.\n\nThis provision gives the federal judiciary, specifically the Supreme Court, the authority to hear and resolve disputes between states, ensuring a peaceful mechanism for addressing interstate conflicts within the constitutional framework. By providing this judicial avenue, the Constitution aims to prevent potential interstate tensions from escalating into more serious confrontations.\n\nThe placement of this provision in Article III, which establishes the judicial branch, underscores the Founders\' intent to use the court system as a neutral arbiter for resolving interstate disputes, rather than leaving such conflicts to be settled through potentially more volatile means.', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=70545, output_tokens=213)
Time: 70.06747603416443s

[TextBlock(text='Based on the Preamble text in the document, the six goals outlined are:\n\n1. To form a more perfect Union\n2. To establish Justice\n3. To insure domestic Tranquility\n4. To provide for the common defense\n5. To promote the general Welfare\n6. To secure the Blessings of Liberty to ourselves and our Posterity\n\nThese goals are stated in the opening paragraph: "We the People of the United States, in Order to form a more perfect Union, establish Justice, insure domestic Tranquility, provide for the common defence, promote the general Welfare, and secure the Blessings of Liberty to ourselves and our Posterity, do ordain and establish this Constitution for the United States of America."', type='text')]
Cache usage: Usage(cache_creation_input_tokens=70520, cache_read_input_tokens=0, input_tokens=29, output_tokens=166)
Time: 13.447766065597534s
[TextBlock(text="Based on Article I of the Constitution, here's a detailed breakdown of the composition and powers of the House of Representatives and Senate:\n\nHouse of Representatives Composition:\n1. Elected every two years by the people of the states\n2. Members must be:\n- At least 25 years old\n- A U.S. citizen for at least 7 years\n- An inhabitant of the state they represent\n3. Number of representatives determined by state population\n4. Initially set with specific numbers for each state until first census\n\nHouse of Representatives Powers:\n1. Sole power of impeachment\n2. Elect their own Speaker and other officers\n3. All bills for raising revenue must originate in the House\n4. Judge the elections, returns, and qualifications of its own members\n\nSenate Composition:\n1. Originally chosen by state legislatures (changed by 17th Amendment)\n2. Two senators per state\n3. Senators serve six-year terms\n4. Members must be:\n- At least 30 years old\n- A U.S. citizen for at least 9 years\n- An inhabitant of the state they represent\n\nSenate Powers:\n1. Sole power to try all impeachments\n2. Confirm presidential appointments\n3. Ratify treaties (requires 2/3 vote)\n4. Elect their own officers, including a President pro tempore\n5. Judge the elections, returns, and qualifications of its own members\n\nShared Congressional Powers:\n- Legislate on matters outlined in Article I, Section 8\n- Override presidential vetoes with 2/3 vote\n- Propose and pass legislation", type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=70520, input_tokens=25, output_tokens=362)
Time: 8.440284967422485s
[TextBlock(text='Based on Article V of the Constitution, the process for proposing and ratifying amendments involves two potential paths:\n\nProposal Methods:\n1. Congressional Proposal: When two-thirds of both Houses of Congress deem it necessary, they can propose amendments.\n\n2. State-Initiated Convention: When two-thirds of the state legislatures request it, Congress shall call a convention for proposing amendments.\n\nRatification Methods:\nAmendments can be ratified through either of two methods:\n1. By legislatures of three-fourths of the states\n2. By conventions in three-fourths of the states, as Congress may prescribe\n\nAdditional Restrictions:\n- Until 1808, no amendment could affect certain clauses related to slavery in Article I, Section 9\n- No state can be deprived of its equal suffrage in the Senate without its consent\n\nThe text explicitly states: "The Congress, whenever two thirds of both Houses shall deem it necessary, shall propose Amendments to this Constitution, or, on the Application of the Legislatures of two thirds of the several States, shall call a Convention for proposing Amendments, which, in either Case, shall be valid to all Intents and Purposes, as Part of this Constitution, when ratified by the Legislatures of three fourths of the several States, or by Conventions in three fourths thereof, as the one or the other Mode of Ratification may be proposed by the Congress."', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=70520, input_tokens=25, output_tokens=315)
Time: 6.593557119369507s
[TextBlock(text='Based on Article I, Section 8, Congress is explicitly granted the following powers:\n\n1. To lay and collect taxes, duties, imposts, and excises (uniformly throughout the United States)\n2. To borrow money on the credit of the United States\n3. To regulate commerce with foreign nations, among the states, and with Indian tribes\n4. To establish a uniform rule of naturalization and uniform bankruptcy laws\n5. To coin money and regulate its value\n6. To provide for the punishment of counterfeiting\n7. To establish post offices and post roads\n8. To promote the progress of science and useful arts by securing copyright and patent rights\n9. To constitute tribunals inferior to the Supreme Court\n10. To define and punish piracies and felonies committed on the high seas and offenses against the law of nations\n11. To declare war, grant letters of marque and reprisal, and make rules concerning captures\n12. To raise and support armies (with appropriations limited to two years)\n13. To provide and maintain a navy\n14. To make rules for the government and regulation of land and naval forces\n15. To provide for calling forth the militia to execute laws, suppress insurrections, and repel invasions\n16. To provide for organizing, arming, and disciplining the militia\n17. To exercise exclusive legislation over the seat of government and federal property\n18. To make all laws necessary and proper for executing the foregoing powers and other powers vested in the government', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=70520, input_tokens=25, output_tokens=325)
Time: 61.858365058898926s
[TextBlock(text='Based on the text of Article III in the Constitution, disputes between states are addressed in Section 2, Clause 1, which states that the judicial power of the United States shall extend to "Controversies between two or more States".\n\nSpecifically, the clause reads: "The judicial power shall extend to all Cases, in Law and Equity, arising under this Constitution, the Laws of the United States, and Treaties made, or which shall be made, under their Authority;--to all Cases affecting Ambassadors, other public Ministers and Consuls;--to all Cases of admiralty and maritime Jurisdiction;--to Controversies to which the United States will be a party;--to Controversies between two or more States;--between a State and Citizens of another State;--between Citizens of different States,--between Citizens of the same State claiming Lands under Grants of different States, and between a State, or the Citizens thereof, and foreign States, Citizens or Subjects."\n\nFurthermore, in Section 2, Clause 2, it specifies that for cases involving states, "the supreme Court shall have original Jurisdiction." This means that the Supreme Court would have the first and primary authority to hear and resolve disputes directly between states.\n\nThe Constitution does not provide detailed procedural guidelines for how such interstate controversies should be resolved, but establishes the Supreme Court as the constitutional venue for addressing these disputes.', type='text')]
Cache usage: Usage(cache_creation_input_tokens=0, cache_read_input_tokens=70520, input_tokens=25, output_tokens=299)
Time: 7.58781623840332s

Sonnet results: